In [1]:
import json

with open("all_places.json", "r", encoding="utf-8") as f:
    all_places = json.load(f)

In [2]:
import re
import html

def clean_review(text):
    if not isinstance(text, str):
        return None

    text = html.unescape(text)
    text = text.strip()

    if len(text) < 3:
        return None

    if not re.search("[가-힣]", text):  # 한글 없는 외국어 리뷰 제거
        return None

    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # 반복 문자 정리
    text = re.sub(r"[^\w\s가-힣.,!?]", "", text)  # 이모지, 특수문자 제거
    text = text[:300]  # 너무 긴 리뷰 자르기

    return text if text else None

for place in all_places:
    original_reviews = place.get("reviews", [])
    cleaned_reviews = []

    for review in original_reviews:
        cleaned = clean_review(review)
        if cleaned:
            cleaned_reviews.append(cleaned)

    place["reviews"] = cleaned_reviews  # ✅ 전처리된 리뷰로 덮어쓰기


In [ ]:
print("여행에서의 희망 키워드를 입력하세요! 예 : 전망대 공원 자연")
"""keyword_hope = []
for i in range(10):
    keyword = input()
    if (keyword == "종료"):
        break
    keyword_hope.append(keyword)"""

print("여행에서의 비희망 키워드를 입력하세요! 예 : 시끄러움 혼잡")
"""keyword_nonhope = []
for i in range(10):
    keyword = input()
    if (keyword == "종료"):
        break
    keyword_nonhope.append(keyword)"""

여행에서의 희망 키워드를 입력하세요! 종료 : 종료
여행에서의 비희망 키워드를 입력하세요! 종료 : 종료


In [60]:
from sentence_transformers import SentenceTransformer
import numpy as np

# ✅ 모델 불러오기 (한 줄로 끝)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
from sklearn.metrics.pairwise import cosine_similarity

def get_sbert_embedding(text):
    if not isinstance(text, str) or not text.strip():
        return np.zeros(model.get_sentence_embedding_dimension())
    return model.encode(text, convert_to_numpy=True)

def get_sbert_review_vector(reviews):
    embeddings = [
        get_sbert_embedding(review)
        for review in reviews if review.strip()
    ]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.get_sentence_embedding_dimension())

def get_place_vector_with_name(place, review_weight=1.0, name_weight=0):
    reviews = place.get("reviews", [])
    name = place.get("name", "")

    review_vec = get_sbert_review_vector(reviews)
    name_vec = get_sbert_embedding(name)

    total_weight = review_weight + name_weight
    final_vec = (review_weight * review_vec + name_weight * name_vec) / total_weight
    return final_vec

# 🔁 문맥 벡터 생성 및 저장
for place in all_places:
    if "reviews" in place and "name" in place:
        place["review_vector"] = get_place_vector_with_name(place)


In [61]:
keyword_hope = ["루프탑","야경","전망대","고층건물","트랜드 카페","파티","바","주류","공원","한강 뷰"]
keyword_nonhope = ["벌레","먼지","혼잡","더러움"]

def clean_keyword(text):
    if not isinstance(text, str):
        return None

    text = html.unescape(text)
    text = text.strip()

    if not re.search("[가-힣]", text):  # 한글 없는 외국어 리뷰 제거
        return None

    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # 반복 문자 정리
    text = re.sub(r"[^\w\s가-힣.,!?]", "", text)  # 이모지, 특수문자 제거
    text = text[:300]  # 너무 긴 리뷰 자르기

    return text if text else None

keyword_hope = [clean_keyword(r) for r in keyword_hope if r]
keyword_nonhope = [clean_keyword(r) for r in keyword_nonhope if r]

In [62]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import defaultdict
import numpy as np

# 1. 희망 키워드 임베딩
hope_embeddings = [get_sbert_embedding(k) for k in keyword_hope if k]

# 2. 최적 K 탐색
best_k = 2
best_score = -1
for k in range(2, min(len(hope_embeddings), 6)):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto").fit(hope_embeddings)
    score = silhouette_score(hope_embeddings, kmeans.labels_)
    if score > best_score:
        best_score = score
        best_k = k

# 3. 최적 K로 클러스터링 수행
final_kmeans = KMeans(n_clusters=best_k, random_state=42, n_init="auto").fit(hope_embeddings)
labels = final_kmeans.labels_

clustered_keywords_list = [[] for _ in range(best_k)]
cluster_centroids = []

# 클러스터 ID별 키워드와 벡터 수집
cluster_vectors = [[] for _ in range(best_k)]

for keyword, label in zip(keyword_hope, labels):
    clustered_keywords_list[label].append(keyword)
    cluster_vectors[label].append(get_sbert_embedding(keyword))  # 각 키워드의 벡터 저장

# 클러스터별 평균 벡터 계산
for vectors in cluster_vectors:
    if vectors:
        centroid = np.mean(vectors, axis=0)
    else:
        centroid = np.zeros(model.get_sentence_embedding_dimension())
    cluster_centroids.append(centroid)

# 결과 저장
keyword_hope = clustered_keywords_list  # [[cluster1 키워드들], [cluster2 키워드들], ...]
keyword_hope_centroids = cluster_centroids  # [cluster1 벡터, cluster2 벡터, ...]

print(keyword_hope)
print(keyword_nonhope)

# 비선호 키워드 임베딩
nonhope_embeddings = [get_sbert_embedding(k) for k in keyword_nonhope if k]

# 평균 임베딩 (빈 경우 대비)
if nonhope_embeddings:
    nonhope_mean_vector = np.mean(nonhope_embeddings, axis=0)
else:
    nonhope_mean_vector = np.zeros(model.get_sentence_embedding_dimension())


c:\Users\hyi1102\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\hyi1102\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\hyi1102\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\hyi1102\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak o

[['트랜드 카페', '파티', '바', '주류'], ['야경', '전망대', '공원', '한강 뷰'], ['루프탑', '고층건물']]
['벌레', '먼지', '혼잡', '더러움']


In [63]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_cluster_scores(review_vector, name_vector, cluster_centroids, alpha=0.2):
    scores = []
    for centroid in cluster_centroids:
        # 각 유사도 개별 계산
        sim_review = (
            cosine_similarity([review_vector], [centroid])[0][0]
            if np.linalg.norm(review_vector) > 0 and np.linalg.norm(centroid) > 0
            else 0.0
        )
        sim_name = (
            cosine_similarity([name_vector], [centroid])[0][0]
            if np.linalg.norm(name_vector) > 0 and np.linalg.norm(centroid) > 0
            else 0.0
        )
        # 이름과 리뷰의 가중합
        score = (1 - alpha) * sim_review + alpha * sim_name
        scores.append(round(score, 4))
    return scores


# 모든 장소에 대해 클러스터 점수 부여
for place in all_places:
    review_vec = place.get("review_vector")
    name_vec = get_sbert_embedding(place.get("name", ""))
    if review_vec is not None:
        place["cluster_scores"] = compute_cluster_scores(
            review_vec, name_vec, keyword_hope_centroids, alpha=0.2
        )

In [64]:
from collections import defaultdict

# 타입별 그룹핑
type_grouped = defaultdict(list)
for place in all_places:
    if "cluster_scores" in place:
        type_grouped[place["type"]].append(place)

# 정렬 및 출력
for place_type, places in type_grouped.items():
    print(f"\n📂 Type: {place_type}")
    sorted_places = sorted(places, key=lambda x: max(x["cluster_scores"]), reverse=True)
    for p in sorted_places[:10]:  # 상위 10개만 출력
        scores_str = ", ".join(f"{s:.2f}" for s in p["cluster_scores"])
        print(f"  - {p['name']} | 점수: [{scores_str}]")



📂 Type: tourist_attraction
  - 흑석동공원 | 점수: [0.24, 0.61, 0.23]
  - 산기슭공원 | 점수: [0.13, 0.55, 0.22]
  - 서래섬 | 점수: [0.19, 0.54, 0.18]
  - 마을숲공원 | 점수: [0.21, 0.53, 0.18]
  - 원효대교 | 점수: [0.22, 0.52, 0.35]
  - 가로공원 | 점수: [0.28, 0.52, 0.29]
  - 덕수공원 | 점수: [0.20, 0.48, 0.23]
  - 거리공원 | 점수: [0.12, 0.48, 0.12]
  - 계남제1근린공원 | 점수: [0.13, 0.48, 0.15]
  - 관악산 자연공원 | 점수: [0.12, 0.46, 0.09]

📂 Type: cafe
  - 로얄호스트 | 점수: [0.49, 0.42, 0.32]
  - 스무디킹 영등포타임스퀘어점 | 점수: [0.44, 0.24, 0.16]
  - 동작노을카페 | 점수: [0.26, 0.43, 0.25]
  - 투썸플레이스 서강대점 | 점수: [0.41, 0.33, 0.19]
  - 할리스 커피 구로점 | 점수: [0.40, 0.25, 0.21]
  - 투썸플레이스 가산디지털점 | 점수: [0.37, 0.24, 0.14]
  - 신촌미플 | 점수: [0.36, 0.30, 0.14]
  - bar sting | 점수: [0.35, 0.27, 0.17]
  - 비하인드 | 점수: [0.35, 0.23, 0.11]
  - 어반트리 | 점수: [0.34, 0.35, 0.22]

📂 Type: bar
  - 카스타운 | 점수: [0.46, 0.29, 0.18]
  - 와바 여의도 직영점 | 점수: [0.43, 0.31, 0.18]
  - 생활맥주 구로디지털점 | 점수: [0.42, 0.26, 0.16]
  - 치어스영등포구청점 | 점수: [0.42, 0.35, 0.21]
  - 통파이브 당산점 | 점수: [0.39, 0.25, 0.18]
  - 푸른유월 | 점수: [0.39, 0

In [69]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# 이름과 리뷰를 함께 고려한 벡터 (가중치 조절 가능)
def get_combined_place_vector(place, review_weight=1.0, name_weight=1.0):
    review_vec = place.get("review_vector", np.zeros(model.get_sentence_embedding_dimension()))
    name_vec = get_sbert_embedding(place.get("name", ""))
    
    if np.linalg.norm(review_vec) == 0 and np.linalg.norm(name_vec) == 0:
        return np.zeros(model.get_sentence_embedding_dimension())
    
    total_weight = review_weight + name_weight
    return (review_weight * review_vec + name_weight * name_vec) / total_weight

# 장소별 비선호 유사도 계산 (이름 포함)
for place in all_places:
    combined_vec = get_combined_place_vector(place, review_weight=1.0, name_weight=1.0)
    if np.linalg.norm(combined_vec) > 0 and np.linalg.norm(nonhope_mean_vector) > 0:
        score = cosine_similarity([combined_vec], [nonhope_mean_vector])[0][0]
        place["nonhope_score"] = round(score, 4)
    else:
        place["nonhope_score"] = 0.0

# 타입별로 그룹화 및 출력
type_to_places = defaultdict(list)
for place in all_places:
    type_to_places[place["type"]].append(place)

print("\n비선호 키워드 유사도 (타입별 상위 3개씩):")
for place_type, places in type_to_places.items():
    print(f"\n🔹 {place_type.title()}:")
    top_places = sorted(places, key=lambda x: x["nonhope_score"], reverse=True)[:10]
    for i, place in enumerate(top_places, 1):
        print(f"{i}. {place['name']}")
        print(f"비선호 유사도: {place['nonhope_score']:.4f} | 평점: {place['rating']} | 리뷰 수: {place['user_ratings_total']}")



비선호 키워드 유사도 (타입별 상위 3개씩):

🔹 Tourist_Attraction:
1. 가로공원
비선호 유사도: 0.3953 | 평점: 3.5 | 리뷰 수: 49
2. 매화근린공원
비선호 유사도: 0.3765 | 평점: 4.1 | 리뷰 수: 29
3. 금천교
비선호 유사도: 0.3528 | 평점: 3.9 | 리뷰 수: 169
4. 방화쌈지공원
비선호 유사도: 0.3411 | 평점: 4.1 | 리뷰 수: 167
5. 서래섬
비선호 유사도: 0.3358 | 평점: 4.4 | 리뷰 수: 762
6. 흑석동공원
비선호 유사도: 0.3261 | 평점: 4.4 | 리뷰 수: 97
7. 덕수공원
비선호 유사도: 0.3092 | 평점: 3.7 | 리뷰 수: 51
8. 이화여자대학교 자연사박물관
비선호 유사도: 0.3009 | 평점: 4.4 | 리뷰 수: 111
9. 돈의문(서대문) 터
비선호 유사도: 0.2893 | 평점: 3.9 | 리뷰 수: 51
10. 부천원미공원
비선호 유사도: 0.2827 | 평점: 4.3 | 리뷰 수: 289

🔹 Cafe:
1. 탐앤탐스 영등포역점
비선호 유사도: 0.4440 | 평점: 3.8 | 리뷰 수: 130
2. 신촌미플
비선호 유사도: 0.4241 | 평점: 4.1 | 리뷰 수: 91
3. 어반트리
비선호 유사도: 0.4031 | 평점: 4 | 리뷰 수: 1
4. 커피빈 홍대역점
비선호 유사도: 0.3678 | 평점: 4 | 리뷰 수: 556
5. 아리스타커피 등촌점
비선호 유사도: 0.3416 | 평점: 3.7 | 리뷰 수: 14
6. 비하인드
비선호 유사도: 0.3274 | 평점: 4.3 | 리뷰 수: 63
7. 이디야커피 서여의도점
비선호 유사도: 0.3260 | 평점: 4 | 리뷰 수: 116
8. 할리스 커피 구로점
비선호 유사도: 0.3207 | 평점: 3.7 | 리뷰 수: 88
9. 핸드픽트호텔
비선호 유사도: 0.3183 | 평점: 4.1 | 리뷰 수: 599
10. 파스쿠찌보라매공원점
비선호 유사도: 0.3131 

In [70]:
import json
import numpy as np

def convert_place_for_json(place):
    p = place.copy()
    for key in ["review_vector", "name_vector"]:
        if isinstance(p.get(key), np.ndarray):
            p[key] = p[key].tolist()
    if "cluster_scores" in p:
        p["cluster_scores"] = list(map(float, p["cluster_scores"]))
    if "nonhope_score" in p:
        p["nonhope_score"] = float(p["nonhope_score"])
    return p

json_ready = [convert_place_for_json(p) for p in all_places]

with open("all_places_embedding.json", "w", encoding="utf-8") as f:
    json.dump(json_ready, f, ensure_ascii=False, indent=2)

print("✅ 저장 완료")

✅ 저장 완료


In [ ]:
import json

with open("all_places_embedding.json", "r", encoding="utf-8") as f:
    all_places = json.load(f)

In [2]:
import math
from datetime import time, datetime, timedelta

# 유클리드 기반 거리 계산 함수
def euclidean(lat1, lon1, lat2, lon2):
    return math.sqrt((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2)

# 거리 계산 함수
def compute_distance_matrix(places): 
    n = len(places)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                matrix[i][j] = euclidean(
                    places[i]['lat'], places[i]['lng'],
                    places[j]['lat'], places[j]['lng']
                )
    return matrix

# 분단위 시간 차이 계산 함수
def time_diff_minutes(t1, t2):
    dt1 = datetime.combine(datetime.today(), t1)
    dt2 = datetime.combine(datetime.today(), t2)
    return abs((dt1 - dt2).total_seconds() / 60)

time_table = []

class ScheduleItem:
    def __init__(self, title, start, end, place_type):
        self.title = title
        self.start = start  
        self.end = end     
        self.place_type = place_type

# 명소 제약 조건 대입
def get_constraints(base_mode="명소 중심"):
    constraints = {
        "must_visit_attraction_every_minutes": 180,  # 3시간 : 마지막 명소 방문 후 일정 시간이 지나면 반드시 새로운 명소를 추천해야 한다는 제약
        "attraction_required": True, # 하루 일정에 명소가 반드시 포함되어야 하는지 여부

        "min_minutes_between_meals": 360,  # 6시간 : 식사 후 식사를 반드시 해야하는 제약 여부
        "require_meal_after_threshold": True, # 식사를 하지 않고 일정 시간이 지나면, 다음 일정으로 반드시 식사를 포함해야 한다는 조건
        "dont_eat_meal": 180, #식사 후 지나가야하는 시간 여부

        "department_store_required_interval": None, # 일정이 특정 시간(분 단위) 이상 지날 때마다 백화점이나 쇼핑몰을 꼭 일정에 넣어야 한다는 제약 조건

        "allow_multiple_cafes": False # 하루 일정에 카페(또는 유사 장소: 빵집 등)를 연속으로 포함하는 것을 허용할지 여부
    }
    # 추가 선택 옵션 반영
    if base_mode =="식사 중심": # 명소 필수 제거하고 식사 가게 제약을 3시간으로 감소
        constraints["attraction_required"] = False
        constraints["min_minutes_between_meals"] = 180

    if base_mode =="카페, 빵집 중심": # 식사 필수 조건 제거하고 명소 필수 제거, 카페연속 허용
        constraints["require_meal_after_threshold"] = False
        constraints["attraction_required"] = False
        constraints["allow_multiple_cafes"] = True

    if base_mode =="쇼핑 중심": # 명소조건 제거 및 백화점 개수 제한 해제
        constraints["department_store_required_interval"] = 180  # 3시간마다 쇼핑
        constraints["attraction_required"] = False

    return constraints

# 인덱스 기준 지난 최대 시간 반환
def get_elapsed_minutes_since_last_type(place_type, time_table, idx):
    """현재 인덱스 기준, 해당 타입(place_type)의 마지막 방문으로부터 경과한 시간(분)을 반환"""
    current_start = time_table[idx].start

    for i in range(idx - 1, -1, -1):  # 현재 index 이전만 탐색
        if time_table[i].place_type == place_type:
            last_end = time_table[i].end
            return time_diff_minutes(last_end, current_start)

    return None  # 해당 타입이 아예 없었던 경우

# 타입 선택 함수
def select_allowed_types(time_table, base_mode, idx):
    allowed_types = ['tourist_attraction', 'cafe', 'restaurant', 'bakery', 'bar', 'shopping_mall']

    constraints = get_constraints(base_mode)

    if constraints["attraction_required"] == True:
        if get_elapsed_minutes_since_last_type('tourist_attraction', time_table, idx)>=constraints["must_visit_attraction_every_minutes"]:
            allowed_types = ['tourist_attraction']
            return allowed_types
    
    if constraints["require_meal_after_threshold"] == True:
        if get_elapsed_minutes_since_last_type('restaurant', time_table, idx)<=constraints["dont_eat_meal"]:
            allowed_types.remove("restaurant")
        if get_elapsed_minutes_since_last_type('restaurant', time_table, idx)>=constraints["min_minutes_between_meals"]:
            allowed_types = ['restaurant']
            return allowed_types
    
    if constraints["department_store_required_interval"] != None:
        if get_elapsed_minutes_since_last_type('shopping_mall', time_table, idx)<=constraints["department_store_required_interval"]:
            allowed_types = ['shopping_mall']
            return allowed_types
    
    if constraints["allow_multiple_cafes"] == False:
        if time_table[idx-1].place_type == "cafe" or time_table[idx-1].place_type == "bakery":
            for t in ["cafe", "bakery"]:
                allowed_types.remove(t)
    
    return allowed_types

def generate_empty_slots(time_table, day_start=time(9, 0), day_end=time(23, 59)):
    empty_slots = []

    def to_datetime(t):
        return datetime.combine(datetime.today(), t)

    # 정렬된 타임테이블로 가정
    sorted_table = sorted(time_table, key=lambda x: x.start)

    # Step 1. 처음 ~ 첫 일정 전 구간
    if not sorted_table or sorted_table[0].start > day_start:
        empty_slots += split_empty_range(day_start, sorted_table[0].start if sorted_table else day_end)

    # Step 2. 일정 사이 빈 공간 찾기
    for i in range(len(sorted_table) - 1):
        current_end = sorted_table[i].end
        next_start = sorted_table[i + 1].start
        if current_end < next_start:
            empty_slots += split_empty_range(current_end, next_start)

    # Step 3. 마지막 일정 ~ 하루 끝
    if sorted_table and sorted_table[-1].end < day_end:
        empty_slots += split_empty_range(sorted_table[-1].end, day_end)

    return empty_slots

def split_empty_range(start_time, end_time):
    slots = []
    dt_start = datetime.combine(datetime.today(), start_time)
    dt_end = datetime.combine(datetime.today(), end_time)
    gap_minutes = int((dt_end - dt_start).total_seconds() // 60)

    if gap_minutes < 90:
        return []  # 너무 짧은 공간은 무시

    elif gap_minutes < 180:
        # 하나로 통짜 슬롯
        slots.append(ScheduleItem(None, start_time, end_time, None))
    else:
        # 90분 단위로 쪼갬
        dt_cursor = dt_start
        while (dt_end - dt_cursor).total_seconds() >= 90 * 60:
            dt_next = dt_cursor + timedelta(minutes=90)
            slots.append(ScheduleItem(None, dt_cursor.time(), dt_next.time(), None))
            dt_cursor = dt_next

        # 남은 시간 (예: 2시간 30분에서 마지막 30분)
        if dt_cursor < dt_end:
            slots.append(ScheduleItem(None, dt_cursor.time(), dt_end.time(), None))

    return slots

In [4]:
time_table = [
    ScheduleItem("경복궁", time(10, 0), time(11, 30), "tourist_attraction"),
    ScheduleItem("점심", time(13, 0), time(14, 0), "restaurant"),
    ScheduleItem("N타워", time(16, 0), time(18, 0), "tourist_attraction"),
]

# 1. 빈 슬롯 생성
empty_slots = generate_empty_slots(time_table)

# 2. 실제 일정과 빈 슬롯을 합침
full_schedule = time_table + empty_slots

# 3. 시작 시간 기준으로 정렬
full_schedule.sort(key=lambda x: x.start)

# 4. 출력
for item in full_schedule:
    title = item.title if item.title else "(빈 슬롯)"
    type_ = item.place_type if item.place_type else "-"
    print(f"{item.start.strftime('%H:%M')} - {item.end.strftime('%H:%M')} | {title} | {type_}")

10:00 - 11:30 | 경복궁 | tourist_attraction
11:30 - 13:00 | (빈 슬롯) | -
13:00 - 14:00 | 점심 | restaurant
14:00 - 16:00 | (빈 슬롯) | -
16:00 - 18:00 | N타워 | tourist_attraction
18:00 - 19:30 | (빈 슬롯) | -
19:30 - 21:00 | (빈 슬롯) | -
21:00 - 22:30 | (빈 슬롯) | -
22:30 - 23:59 | (빈 슬롯) | -
